In [ ]:
!python3 --version

Python 3.6.9


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%cd gdrive/My Drive/BERTweet-ner

In [ ]:
!pip3 install datasets
!pip3 install transformers
!pip3 install -U sentence-transformers

In [ ]:
!pip3 install emoji

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import copy

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "vinai/bertweet-base"
batch_size = 16

In [ ]:
# # reading the training set
# f = open("wnut17_train.annotated", "r")
# file_text=f.read()
# df_columns=['id','tokens','ner_tags']
# df_holder=[]
# bio_sentences=list(filter (lambda elem: elem!='', file_text.split('\n\n')))

# for bio_id, bio_sentence in enumerate(bio_sentences):
    
#     words=[]
#     annotations=[]
#     lines=bio_sentence.split('\n')

#     for line in lines:
#         if(line):
#             tabs=line.split('\t')
#             if not((tabs[0]=='')&(tabs[1]=='')):
#                 word=tabs[0]
#                 tag=tabs[1]
#                 # if('-' not in tabs[1]):
#                 #     tag=tabs[1]
#                 # else:
#                 #     tag=tabs[1].split('-')[0]
#                 if(word.strip().startswith('https:')):
#                     word='@url'
#                 words.append(word.strip())
#                 annotations.append(tag)

#     # text=words.strip()
#     # annotation=','.join(get_entities(word_tag_tuples_annotation))
#     df_dict={'ID':str(bio_id),'tokens':words,'ner_tags':annotations}
#     df_holder.append(df_dict)
# df_train = pd.DataFrame(df_holder,columns=df_columns)
# print(len(df_train))
# df_train.to_csv("wnut17train.csv", sep=',', encoding='utf-8',index=False)

# # reading the validation set
# f = open("wnut17_validation.annotated", "r")
# file_text=f.read()
# df_columns=['id','tokens','ner_tags']
# df_holder=[]
# bio_sentences=list(filter (lambda elem: elem!='', file_text.split('\n\n')))

# for bio_id, bio_sentence in enumerate(bio_sentences):
    
#     words=[]
#     annotations=[]
#     lines=bio_sentence.split('\n')

#     for line in lines:
#         if(line):
#             tabs=line.split('\t')
#             if not((tabs[0]=='')&(tabs[1]=='')):
#                 word=tabs[0]
#                 tag=tabs[1]
#                 # if('-' not in tabs[1]):
#                 #     tag=tabs[1]
#                 # else:
#                 #     tag=tabs[1].split('-')[0]
#                 if(word.strip().startswith('https:')):
#                     word='@url'
#                 words.append(word.strip())
#                 annotations.append(tag)

#     # text=words.strip()
#     # annotation=','.join(get_entities(word_tag_tuples_annotation))
#     df_dict={'ID':str(bio_id),'tokens':words,'ner_tags':annotations}
#     df_holder.append(df_dict)
# df_validation = pd.DataFrame(df_holder,columns=df_columns)
# print(len(df_validation))
# df_validation.to_csv("wnut17validation.csv", sep=',', encoding='utf-8',index=False)

# # reading the test set
# f = open("wnut17_test.annotated", "r")
# file_text=f.read()
# df_columns=['id','tokens','ner_tags']
# df_holder=[]
# bio_sentences=list(filter (lambda elem: elem!='', file_text.split('\n\n')))

# for bio_id, bio_sentence in enumerate(bio_sentences):
    
#     words=[]
#     annotations=[]
#     lines=bio_sentence.split('\n')

#     for line in lines:
#         if(line):
#             tabs=line.split('\t')
#             if not((tabs[0]=='')&(tabs[1]=='')):
#                 word=tabs[0]
#                 tag=tabs[1]
#                 # if('-' not in tabs[1]):
#                 #     tag=tabs[1]
#                 # else:
#                 #     tag=tabs[1].split('-')[0]
#                 if(word.strip().startswith('https:')):
#                     word='@url'
#                 words.append(word.strip())
#                 annotations.append(tag)

#     # text=words.strip()
#     # annotation=','.join(get_entities(word_tag_tuples_annotation))
#     df_dict={'ID':str(bio_id),'tokens':words,'ner_tags':annotations}
#     df_holder.append(df_dict)
# df_test = pd.DataFrame(df_holder,columns=df_columns)
# print(len(df_test))
# df_test.to_csv("wnut17test.csv", sep=',', encoding='utf-8',index=False)

In [ ]:

# train_dataset = load_dataset('csv', data_files="wnut17train.csv")
# validation_dataset = load_dataset('csv', data_files='wnut17validation.csv')
# test_dataset = load_dataset('csv', data_files="wnut17test.csv")

datasets = load_dataset("wnut_17")

Dataset wnut_17 downloaded and prepared to /root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/983205ce50100b6e8fce4b3d402f36dce9b206f736e1a630c78fb25e1d23b9e8. Subsequent calls will reuse this data.


In [ ]:
# train_dataset
datasets
# train_dataset["train"].features[f"ner_tags"]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
label_all_tokens = True

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
# example = datasets["train"][4]
# print(example["tokens"])
# tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
# # input_ids = tokenizer.encode(example["tokens"])
# # tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# print(tokenized_input)
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# print(tokens)
# # len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])
# print({x : tokenizer.encode(x, add_special_tokens=False) for x in example["tokens"]})

In [ ]:
expanded_label_dict={0:'O', 1:'B-corporation', 2:'I-corporation', 3:'B-creative-work', 4:'I-creative-work', 5:'B-group', 6:'I-group', 7:'B-location', 8:'I-location', 9:'B-person', 10:'I-person', 11:'B-product', 12:'I-product'}
BIO_dict={'O':0,'B':1,'I':2}

def tokenize_and_align_labels(example):
        
    tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
    inputId_to_token_dict={}
    for index, token in enumerate(example["tokens"]):
        values=tokenizer.encode(token, add_special_tokens=False, truncation=True)
        for value in values:
            try:
                inputId_to_token_dict[value].append(index)
            except KeyError:
                inputId_to_token_dict[value]=[index]
    labels=[]
    for inputID in tokenized_input['input_ids']:
        try:
            index_list=copy.deepcopy(inputId_to_token_dict[inputID])
            index_to_address=index_list.pop(0)

            # label=BIO_dict[expanded_label_dict[example['ner_tags'][index_to_address]][0]] #Just BIO
            label = example['ner_tags'][index_to_address]

            labels.append(label)
            inputId_to_token_dict[inputID]=index_list
        except KeyError:
            labels.append(-100)

    assert (len(tokenized_input['input_ids']) == len(labels))
    tokenized_input['labels']=labels
    
    return tokenized_input

In [ ]:
# tokenize_and_align_labels(datasets['train'][2])
tokenized_datasets = datasets.map(tokenize_and_align_labels)

In [ ]:
tokenized_datasets['train']

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'token_type_ids', 'tokens'],
    num_rows: 3394
})

In [ ]:
!pip3 install seqeval

     |████████████████████████████████| 51kB 3.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=c27d82ba87592133901d4fc894f3287027400d18db6893a1d8527eb1e37d0e52
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

In [ ]:
# tokenized_datasets['train'].features["labels"]

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names

# label_list = ['O','B','I']

print(label_list)
print(len(label_list))

['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product']
13


In [ ]:
import numpy as np

def compute_metrics(p):
    # print(p.shape)
    output, labels = p

    # print(len(predictions))
    # print(predictions[0].shape)
    # for elem in predictions[1]:
    #   print(elem.shape)

    predictions, _ = output
    
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# config = AutoConfig.from_pretrained(model_checkpoint, output_hidden_states=True)
# config

In [ ]:
alt_model = AutoModelForTokenClassification.from_pretrained("vinai/bertweet-base", output_hidden_states=True, num_labels=len(label_list))
# alt_model.config

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [ ]:
alt_training_args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)
alt_training_args

TrainingArguments(output_dir=test-ner, overwrite_output_dir=False, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Feb18_01-08-11_a065c8063471, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O1, fp16_backend=auto, local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=500, dataloader_num_workers=0, past_index=-1, run_name=test-ner, disable_tqdm=False, remove_unused_columns=True, label_names=None, load_best_model_at_end=False, metric_

In [ ]:
alt_trainer = Trainer(
    alt_model,
    alt_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

**Model output is a tuple, when all hidden states are returned, i.e. output_hidden_states =True in config: (output, hidden-layers)**

In [ ]:
alt_trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,No log,0.382188,0.446198,0.258520,0.327368,0.918466,8.814100,114.476000
2,No log,0.345560,0.678407,0.368246,0.477371,0.929527,4.371600,230.809000
3,0.296100,0.321379,0.634085,0.420615,0.505747,0.935177,4.636100,217.638000


/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=639, training_loss=0.2678769310128894, metrics={'train_runtime': 131.2815, 'train_samples_per_second': 4.867, 'total_flos': 309189764708700, 'epoch': 3.0})

## **Initial Trainer with Default Config**

In [ ]:
# labels = [label_list[i] for i in example["ner_tags"]]
# metric.compute(predictions=[labels], references=[labels])

# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=3) #Just BIO

In [ ]:
# args = TrainingArguments(
#     f"test-{task}",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

In [ ]:
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [ ]:
# trainer.train()

In [ ]:
trainer.evaluate()

NameError: ignored

## **Predict on validation/test**

In [ ]:
# predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
# predictions = np.argmax(predictions, axis=2)


In [ ]:
model_returns, labels, _ = alt_trainer.predict(tokenized_datasets["test"])


/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
predictions, _ = model_returns

In [ ]:
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)

/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
results

{'corporation': {'f1': 0.0, 'number': 118, 'precision': 0.0, 'recall': 0.0},
 'creative-work': {'f1': 0.043795620437956206,
  'number': 236,
  'precision': 0.15789473684210525,
  'recall': 0.025423728813559324},
 'group': {'f1': 0.033783783783783786,
  'number': 257,
  'precision': 0.1282051282051282,
  'recall': 0.019455252918287938},
 'location': {'f1': 0.4812623274161736,
  'number': 217,
  'precision': 0.4206896551724138,
  'recall': 0.5622119815668203},
 'overall_accuracy': 0.9209552512112311,
 'overall_f1': 0.3703448275862069,
 'overall_precision': 0.5536082474226804,
 'overall_recall': 0.2782383419689119,
 'person': {'f1': 0.5437415881561238,
  'number': 885,
  'precision': 0.6722129783693843,
  'recall': 0.45649717514124294},
 'product': {'f1': 0.0, 'number': 217, 'precision': 0.0, 'recall': 0.0}}

In [ ]:
# print(labels)

In [ ]:
# # print(predictions[:5])
# # Remove ignored index (special tokens)
# true_predictions = [
#     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# true_labels = [
#     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# print(true_predictions[:5])
# print(true_labels[:5])

In [ ]:
# results = metric.compute(predictions=true_predictions, references=true_labels)

In [ ]:
# print(results)

## **Pre-processing on custom test set**

In [ ]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [ ]:
import re
import emoji
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import string

string.punctuation=string.punctuation+'…‘’'

In [ ]:
def get_entities(word_tag_tuples):
    
    mentions=[]
    candidateMention=''
    #emoji.get_emoji_regexp().sub(u'', candidateMention)
    for tup in word_tag_tuples:
        candidate=tup[0]
        tag=tup[1]
        if(tag=='O'):
            if(candidateMention):
                if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                    mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                    if mention_to_add.endswith("'s"):
                        li = mention_to_add.rsplit("'s", 1)
                        mention_to_add=''.join(li)
                    elif mention_to_add.endswith("’s"):
                        li = mention_to_add.rsplit("’s", 1)
                        mention_to_add=''.join(li)
                    else:
                        mention_to_add=mention_to_add
                    if(mention_to_add!=''):
                        mentions.append(mention_to_add)
            candidateMention=''
        else:
            if (tag=='B'):
                if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                    mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                    if mention_to_add.endswith("'s"):
                        li = mention_to_add.rsplit("'s", 1)
                        mention_to_add=''.join(li)
                    elif mention_to_add.endswith("’s"):
                        li = mention_to_add.rsplit("’s", 1)
                        mention_to_add=''.join(li)
                    else:
                        mention_to_add=mention_to_add
                    if(mention_to_add!=''):
                        mentions.append(mention_to_add)
                candidateMention=candidate
            else:
                candidateMention+=" "+candidate
        # if (tag=='B'):
        #     if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))):
        #         mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
        #         if(mention_to_add):
        #             mentions.append(mention_to_add)
        #     candidateMention=candidate
        # else:
        #     candidateMention+=" "+candidate
    if(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).strip()):
        if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
            mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
            if(mention_to_add!=''):
                mentions.append(mention_to_add)
        # mentions.append(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip())
    # print('extracted mentions:', mentions)
    return mentions

In [ ]:
def get_encoding_seq(tweet_word_list, mentions):
    print(tweet_word_list)
    print(mentions)
    tweet_word_index=0
    encoded_tag_sequence=[]
    while(mentions):
        current_mention=[token.strip() for token in mentions.pop(0).split(' ')]
        while(normalize(current_mention[0])!=normalize(tweet_word_list[tweet_word_index])):
            encoded_tag_sequence.append('O')
            tweet_word_index+=1
        if(normalize(current_mention[0])==normalize(tweet_word_list[tweet_word_index])):
            for token_index, token in enumerate(current_mention):
                if(token_index==0):
                    encoded_tag_sequence.append('B')
                else:
                    encoded_tag_sequence.append('I')
                tweet_word_index+=1
    while(tweet_word_index<len(tweet_word_list)):
        encoded_tag_sequence.append('O')
        tweet_word_index+=1
        
    print(encoded_tag_sequence)
    return encoded_tag_sequence

In [ ]:
gutenberg_text = ""
for file_id in gutenberg.fileids():
    gutenberg_text += gutenberg.raw(file_id)
tokenizer_trainer = PunktTrainer()
tokenizer_trainer.INCLUDE_ALL_COLLOCS = True
tokenizer_trainer.train(gutenberg_text)

my_sentence_tokenizer = PunktSentenceTokenizer(tokenizer_trainer.get_params())
my_sentence_tokenizer._params.abbrev_types.add('dr')
my_sentence_tokenizer._params.abbrev_types.add('c.j')
my_sentence_tokenizer._params.abbrev_types.add('u.s')
my_sentence_tokenizer._params.abbrev_types.add('u.s.a')

def normalize_to_sentences(text):
    tweetSentences=list(filter (lambda sentence: len(sentence)>1, text.split('\n')))
    tweetSentenceList_inter=custom_flatten(list(map(lambda sentText: my_sentence_tokenizer.tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
    tweetSentenceList=list(filter (lambda sentence: len(sentence)>1, tweetSentenceList_inter))
    return tweetSentenceList

def custom_flatten(mylist, outlist,ignore_types=(str, bytes, int)):
    
    if (mylist !=[]):
        for item in mylist:
            #print not isinstance(item, ne.NE_candidate)
            if isinstance(item, list) and not isinstance(item, ignore_types):
                custom_flatten(item, outlist)
            else:
                item=item.strip(' \t\n\r')
                outlist.append(item)
    return outlist

def preprocess(filename):
    """save a file with token, label and prediction in each row"""
    tweet_to_sentences_w_annotation={}
    sentenceID=0
    test=pd.read_csv("data/"+filename,sep =',',keep_default_na=False)
    # outputfilename="data/covid/covid_2K.txt"
    
    all_annotated_ne=[]
    tweetsentences=[]
    tokenizedsentences=[]
    
    for row in test.itertuples():
        tweetID=str(row.Index)
        text=str(row.TweetText)
        row_sentences = normalize_to_sentences(text)
        tweetsentences += row_sentences
        tokenizedsentences += [tokenizer(sentence, is_split_into_words=True) for sentence in row_sentences]
        # print(text)
        
        mentions=[]
        for sentence_level in str(row.mentions_other).split(';'):
            if(sentence_level):
                for mention in sentence_level.split(','):
                    if(mention):
                        mentions.append(mention.lower().strip(string.punctuation).strip())
        mentions=list(filter(lambda element: ((element !='')&(element !='nan')), mentions))
        # all_annotated_ne.extend(mentions)
        
        # if(row_sentences):
        tweet_to_sentences_w_annotation[tweetID]=((sentenceID,sentenceID+len(row_sentences)),mentions)
        sentenceID+=len(row_sentences)
        # else:
        #     tweet_to_sentences_w_annotation[tweetID]=((sentenceID,sentenceID+1),mentions)
        #     sentenceID+=1
        # print(sentenceID,len(row_sentences))
    return tweetsentences, tokenizedsentences, tweet_to_sentences_w_annotation


In [ ]:
def calculate_f1(tweet_to_sentences_w_annotation, tweetsentences, true_predictions):
    
    # dataset, i = [], 0
    ner_arrays=[]
    file_write_text=''
    all_detected_ne=[]
    all_annotated_ne=[]
    
    for n, tweet in enumerate(tweetsentences):
        # tweet_data = list(zip(tweet, true_labels[n], predictions[i:i + len(tweet)]))
        # word_tag_tuples=zip(tweet,predictions[i:i + len(tweet)])
        
        assert (len(tweet)==len(true_predictions[n]))
        word_tag_tuples=zip(tweet,true_predictions[n])
        entities_from_sentence=get_entities(word_tag_tuples)
        # line_text='\t'.join(entities_from_sentence)
        # file_write_text+=line_text+'\n'
        # print(entities_from_sentence)
        all_detected_ne.extend(entities_from_sentence)
        ner_arrays.append(entities_from_sentence)
        # dataset += tweet_data + [()]
    
    print('tally:',len(tweetsentences),len(ner_arrays))
    system_output_mention_list=list(set(all_detected_ne))
    # file_write_text='\n'.join(system_output_mention_list)
    # f1= open(outputfilename, "w")
    # f1.write(file_write_text)
    # f1.close()
    
    true_positive_count=0
    false_positive_count=0
    false_negative_count=0
    total_mentions=0
    total_annotation=0
    
    
    for tweetID in tweet_to_sentences_w_annotation.keys():
        unrecovered_annotated_mention_list=[]
        tp_counter_inner=0
        fp_counter_inner=0
        fn_counter_inner=0
        
        annotated_mention_list=tweet_to_sentences_w_annotation[tweetID][1]
        all_annotated_ne.extend(annotated_mention_list)
        output_mentions_list=[]
        idRange=tweet_to_sentences_w_annotation[tweetID][0]
        for sentID in range(idRange[0],idRange[1]):
            output_mentions_list+=ner_arrays[sentID]
        print(tweetID,annotated_mention_list,output_mentions_list)
        all_postitive_counter_inner=len(output_mentions_list)
        while(annotated_mention_list):
            if(len(output_mentions_list)):
                annotated_candidate= annotated_mention_list.pop()
                if(annotated_candidate in output_mentions_list):
                    output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                    tp_counter_inner+=1
                else:
                    unrecovered_annotated_mention_list.append(annotated_candidate)
            else:
                unrecovered_annotated_mention_list.extend(annotated_mention_list)
                break
        # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
        fn_counter_inner=len(unrecovered_annotated_mention_list)
        fp_counter_inner=all_postitive_counter_inner - tp_counter_inner
        
        print(tp_counter_inner,fp_counter_inner,fn_counter_inner)
        
        true_positive_count+=tp_counter_inner
        false_positive_count+=fp_counter_inner
        false_negative_count+=fn_counter_inner
        
    print('true_positive_count,false_positive_count,false_negative_count:')
    print(true_positive_count,false_positive_count,false_negative_count)
    
    precision=(true_positive_count)/(true_positive_count+false_positive_count)
    recall=(true_positive_count)/(true_positive_count+false_negative_count)
    f_measure=2*(precision*recall)/(precision+recall)
            
    print('========Entity Mention Detection========')
    print('precision: ',precision)
    print('recall: ',recall)
    print('f_measure: ',f_measure)

    print('========Entity Detection========')
    true_positive_entities =  len(list(set(all_detected_ne).intersection(set(all_annotated_ne))))
    false_positive_entities = len(list(set(all_annotated_ne)-set(all_detected_ne)))
    false_negative_entities = len(list(set(all_detected_ne)-set(all_annotated_ne)))

    precision= (true_positive_entities)/(true_positive_entities+false_positive_entities)
    recall= (true_positive_entities)/(true_positive_entities+false_negative_entities)
    f_measure = 2*(precision*recall)/(precision+recall)

    print('precision: ',precision)
    print('recall: ',recall)
    print('f_measure: ',f_measure)   

    return

In [ ]:
# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/TwiCSv2/data/tweets_3k_annotated.csv",sep =',',keep_default_na=False)
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('tweets_3k_annotated.csv')

# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('venezuela.csv')

# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('billdeblasio.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('pikapika.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('ripcity.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('billnye.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('roevwade.csv')

testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('wnut17test.csv')

In [ ]:
# tokenized_datasets = testset.map(tokenize_and_align_labels)
print(len(testset))
print(testset[:5])

1864
['& gt ; * The soldier was killed when another avalanche hit an army barracks in the northern area of Sonmarg , said a military spokesman .', '& gt ; * Police last week evacuated 80 villagers from Waltengoo Nar where dozens were killed after a series of avalanches hit the area in 2005 in the south of the territory .', '& gt ; * The army on Thursday recovered the bodies of ten of its men who were killed in an avalanche the previous day .', '& gt ; * The four civilians killed included two children of a family whose house was hit by a separate avalanche , also on Wednesday , a police spokesman said .', 'The bodies of the soldiers were recovered after the concerted efforts of the Avalanche Rescue Teams ( ART ) , which is equipped to work in inhospitable terrain and weather conditions .']


In [ ]:
# predictions, labels, _ = trainer.predict(tokenizedtestset)
# predictions = np.argmax(predictions, axis=2)

In [ ]:
# print(len(predictions))
# print(predictions[:5])

In [ ]:
def collate_token_labels(token_dict, prediction_labels):
    counter=0
    collated_labels=[]
    for key in token_dict.keys():
        vals=token_dict[key]
        labels=prediction_labels[counter:counter+len(vals)]
        if('I' in labels):
            collated_labels.append('I')
        elif('B' in labels):
            collated_labels.append('B')
        else:
            collated_labels.append('O')
        counter+=len(vals)
    return collated_labels

In [ ]:
# Set device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [ ]:
predictions=[]
tokenized_sentences=[]
count=0
with torch.no_grad():
    for test_record in testset:
        # print(test_record)
        # test_record=test_record.lower()
        tokenized_input=tokenizer(test_record)
        initial_input_ids = torch.tensor([tokenizer.encode(test_record)])
        token_dict = {x : tokenizer.encode(x, add_special_tokens=False) for x in test_record.split()}
        input_ids = initial_input_ids.to(device)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

        # output = model(input_ids)

        output = alt_model(input_ids)
        
        
        prediction = (torch.argmax(output.logits, axis=2))
        prediction = prediction.cpu().numpy().reshape(-1)
        prediction_labels=[label_list[l].split('-')[0] for l in prediction]
        prediction_labels=collate_token_labels(token_dict, prediction_labels[1:-1])
        predictions.append(prediction_labels)
        tokenized_sentences.append(token_dict.keys())
        assert (len(prediction_labels)==len(token_dict.keys()))

        if(count<=5):
            print(len(output.hidden_states))
            print(token_dict.keys())
            print(initial_input_ids)
            print(input_ids)
            print(prediction_labels)
            print('===============')
        count+=1

print(len(predictions),len(tokenized_sentences))

13
dict_keys(['&', 'gt', ';', '*', 'The', 'soldier', 'was', 'killed', 'when', 'another', 'avalanche', 'hit', 'an', 'army', 'barracks', 'in', 'the', 'northern', 'area', 'of', 'Sonmarg', ',', 'said', 'a', 'military', 'spokesman', '.'])
tensor([[    0,    55,  8083,   208,   110,    47, 10998,    38,  1294,    64,
           347, 36791, 25313,   512,    74,  3466,  2149, 18549,    16,     6,
          7322,  1149,    15,  7725,  2350,   714,     7,   193,    11,  2492,
         34196,     4,     2]])
tensor([[    0,    55,  8083,   208,   110,    47, 10998,    38,  1294,    64,
           347, 36791, 25313,   512,    74,  3466,  2149, 18549,    16,     6,
          7322,  1149,    15,  7725,  2350,   714,     7,   193,    11,  2492,
         34196,     4,     2]], device='cuda:0')
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O']
13
dict_keys(['&', 'gt', ';', '*', 'Police', 'last', 'week', 'evacuated', '

In [ ]:
print(tokenized_sentences[:5])
print(predictions[:5])

In [ ]:
calculate_f1(tweet_to_sentences_w_annotation, tokenized_sentences, predictions)